In [25]:
# Importing all the libraries

In [26]:
import os
import numpy as np
import cv2
from random import randint, choice
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from PIL import Image, ImageOps
from keras import  utils
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing import image
from keras.applications.mobilenet import MobileNet


### O conjunto de dados utilizado é uma versão aumentada do conjunto de dados original, utilizando o ImageDataGenerator.

In [27]:
X_treinamento, Y_treinamento = [], []
tamanho_img = 224
parasitas = ['Entamoeba histolytica', 'Giardia_cyst']
        
for i in parasitas: 
    caminho_dataset = os.path.join(r"dataset_proto_aug\Train", i)
    for j in os.listdir(caminho_dataset):
        img = cv2.imread(os.path.join(caminho_dataset, j)) 
        img = cv2.resize(img, (tamanho_img , tamanho_img )) 
        X_treinamento.append(img) 
        Y_treinamento.append(i)
        
X_train_np = np.array(X_treinamento) 
Y_Train_np = np.array(Y_treinamento)

X_train, Y_train = shuffle(X_train_np, Y_Train_np, random_state = 101)
X_train.shape

(4843, 224, 224, 3)

# Train Test Split

In [28]:
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, random_state = 101, test_size = 0.1)

In [29]:
y_train_new = []
for i in y_train:
    y_train_new.append(parasitas.index(i)) 
y_train = y_train_new
y_train = utils.to_categorical(y_train)

y_test_new = []
for i in y_test:
    y_test_new.append(parasitas.index(i))
y_test = y_test_new
y_test = utils.to_categorical(y_test)

In [30]:
print('Train:', x_train.shape, y_train.shape)
print('Validation:', x_test.shape, y_test.shape)

Train: (4358, 224, 224, 3) (4358, 2)
Validation: (485, 224, 224, 3) (485, 2)


# MobileNet Neural Network

In [31]:
mobile = MobileNet(weights = 'imagenet', include_top = False, input_shape = (224,224,3))

In [32]:
modelo_mobile = Sequential()

modelo_mobile.add(mobile)
modelo_mobile.add(GlobalAveragePooling2D())
modelo_mobile.add(Dense(1024, activation = 'relu'))
modelo_mobile.add(Dense(1024, activation = 'relu'))
modelo_mobile.add(Dense(512, activation = 'relu'))
modelo_mobile.add(Dense(2, activation = 'softmax'))

In [33]:
modelo_mobile.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenet_1.00_224 (Functional) │ (None, 7, 7, 1024)     │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,853,890 (22.33 MB)

 Trainable params: 5,832,002 (22.25 MB)

 Non-trainable params: 21,888 (85.50 KB)

In [34]:
modelo_mobile.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['Accuracy']) 

In [35]:
modelo_mobile_treinado = modelo_mobile.fit(x_train, y_train, epochs = 10,validation_split = 0.2)

Epoch 1/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 113s 950ms/step - Accuracy: 0.8814 - loss: 0.3928 - val_Accuracy: 0.9622 - val_loss: 0.2386
Epoch 2/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 106s 977ms/step - Accuracy: 0.9864 - loss: 0.0415 - val_Accuracy: 0.9782 - val_loss: 0.0651
Epoch 3/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 108s 991ms/step - Accuracy: 0.9899 - loss: 0.0355 - val_Accuracy: 0.9484 - val_loss: 0.2357
Epoch 4/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 107s 983ms/step - Accuracy: 0.9947 - loss: 0.0146 - val_Accuracy: 0.8647 - val_loss: 0.6781
Epoch 5/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 105s 966ms/step - Accuracy: 0.9965 - loss: 0.0096 - val_Accuracy: 1.0000 - val_loss: 6.0656e-05
Epoch 6/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - Accuracy: 0.9926 - loss: 0.0209 - val_Accuracy: 0.9828 - val_loss: 0.0684
Epoch 7/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 114s 1s/step - Accuracy: 0.9819 - loss: 0.0624 - val_Accuracy: 0.9943 - val_loss: 0.0251
Epoch 8/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 106s 968ms/step - Accuracy: 0.9935 - l

In [36]:
modelo_mobile.evaluate(x_test, y_test, batch_size = 100)

5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 630ms/step - Accuracy: 1.0000 - loss: 9.2517e-04


[0.0016626556171104312, 1.0]

# Escolha das imagens e converção


In [48]:
#Escolhe aleatóriamente uma imagem

dir_real = "dataset_protozoarios\\Dataset\\"
escolha_dirParasito = choice(os.listdir(dir_real))
dir_parasito =  os.path.join(dir_real + escolha_dirParasito) 
escolha_img = choice(os.listdir(dir_parasito))
img_final =  os.path.join(dir_parasito, escolha_img)
img_open = Image.open(img_final)

#Redimencionamento
size = (224, 224)
img_rgb = img_open.convert('RGB')
img = ImageOps.fit(img_rgb, size, Image.Resampling.LANCZOS)
img_array = np.array(img)
img_array.shape

(224, 224, 3)

In [49]:
img_array_reshape = img_array.reshape(1, 224, 224, 3)
img_array_reshape.shape

(1, 224, 224, 3)

### MobileNet

In [61]:
d = modelo_mobile.predict(img_array_reshape)
pred_index = np.argmax(d)
confidence = float(d[0][pred_index]) * 100

class_names = ['Entamoeba histolytica', 'Giardia_cyst']  # garanta que a ordem esteja correta
predicted_class = class_names[pred_index]

print(f"Classe prevista: {predicted_class} com confiança de {confidence:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Classe prevista: Entamoeba histolytica com confiança de 100.00%


In [62]:
acc = np.amax(d) * 100
acc = round(acc, 3)
print(acc)

100.0


In [63]:
if(indices == 0):
    print("Entamoeba histolytica")
else:
    print("Giardia cyst")

Entamoeba histolytica


In [53]:
full_img = image.load_img(img_final)
print(img_final)


dataset_protozoarios\Dataset\Entamoeba histolytica\EH44.jpg


# Salvando o Modelo

In [43]:
modelo_mobile.save('MobileNet_model.h5')